<a href="https://colab.research.google.com/github/Darja555/Cleaning-and-analyzing-online-programming-school-data./blob/main/Product_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Работа с данными
import pandas as pd
import numpy as np

# Библиотеки для визуализации
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [ ]:
# Загружаем данные из файла pickle
with open('clean_data.pkl', 'rb') as f:
    data_frames = pd.read_pickle(f)
# Теперь вы можете получить доступ к вашим DataFrame
deals = data_frames['deals']
calls = data_frames['calls']
contacts = data_frames['contacts']
spend = data_frames['spend']
# Пример вывода DataFrame
display(contacts.info(), calls.info(), spend.info(), deals.info())

<class 'pandas.core.frame.DataFrame'>
Index: 18510 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Id                  18510 non-null  object        
 1   Contact Owner Name  18510 non-null  category      
 2   Created Time        18510 non-null  datetime64[ns]
 3   Modified Time       18510 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](2), object(1)
memory usage: 596.7+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 88815 entries, 3 to 95872
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Id                          88815 non-null  int64         
 1   Call Start Time             88815 non-null  datetime64[ns]
 2   Call Owner Name             88815 non-null  category      
 3   CONTACTID                   88815 non-null  object        


None

None

None

None

#Задание по продуктовой аналитике
Найти точку роста бизнеса и сформулировать гипотезу улучшение бизнес процесса
для роста метрик и опишите их механику тестирования с учетом того что тест не
должен занимать больше 2-х недель.
1. Посчитать юнит-экономику по продуктам.
2. Из юнит-экономики определить точки роста бизнеса.
3. Понять дерево метрик для бизнеса.
4. Понять на какую метрику продукта они будут воздействовать и сформировать
гипотезы.
5. Описать метод проверки гипотез с формулированием условия проведения
гипотезы.

## 1. Посчитать юнит-экономику по продуктам (расчет метрик).

* UA (Users) — количество лидов (юнитов масштабирования)
* B (Bayers) — количество клиентов
 C1 (Conversion Rate) = B / UA = конверсия
* AC (Marketing Budget) — маркетинговый бюджет
* CPA (Cost Per Acquisition) = AC / UA = стоимость привлечения одного UA
* CAC (Customer Acquisition Cost) = AC / B = стоимость привлечения одного B
* Gross Revenue (Total Sales) — выручка общая
* AOV (Average Order Value) = Gross Revenue / B = средний чек
* APC (Average Purchases per Customer) — среднее число сделок на одного B
* CLTV (Customer Lifetime Value) = AOV × APC = средняя валовая прибыль на одного UA(CLTV без GOGS и 1sGOGS, так как нет данных)
* LTV (Lifetime Value) = Gross Revenue / UA = средняя валовая прибыль на одного UA
* CM (Contribution Margin) = (CLTV − CAC) × B = маржинальная прибыль
* Total Deals (Total successful of deals) — количество успешных сделок




In [ ]:
# Установка стилей отображения
pd.set_option('display.float_format', '{:,.2f}'.format)

# UA (Users) - количество лидов (юнитов масштабирования)
users = deals.groupby(['Product', 'Education Type'], observed=True)['Id'].nunique().rename('UA')

# B (Bayers) - количество клиентов
bayers = deals[deals['Stage'] == 'Payment Done'].groupby(['Product', 'Education Type'], observed=True)['Id'].nunique().rename('Bayers')

# Total Deals (Total successful deals) - количество успешных сделок
total_deals = deals[deals['Stage'] == 'Payment Done'].groupby(['Product', 'Education Type'], observed=True)['Id'].count().rename('Total Deals')

# Gross Revenue (Total Sales) - выручка общая
gross_revenue = deals[deals['Stage'] == 'Payment Done'].groupby(['Product', 'Education Type'], observed=True)['Offer Total Amount'].sum().rename('Gross Revenue')

# Объединение данных в один DataFrame
merged_df = users.to_frame().join([bayers, total_deals, gross_revenue], how='left').reset_index()

# AC (Marketing Budget) - общий маркетинговый бюджет для всех продуктов
total_ac = spend['Spend'].sum()

# Преобразование числовых столбцов
numeric_columns = ['UA', 'Bayers', 'Total Deals', 'Gross Revenue']
for col in numeric_columns:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Расчет метрик с обработкой деления на ноль
merged_df['C1'] = (merged_df['Bayers'] / merged_df['UA'].replace(0, np.nan)) * 100  # Переводим в проценты
merged_df['AC'] = total_ac
merged_df['CPA'] = merged_df['AC'] / merged_df['UA'].replace(0, np.nan)
merged_df['CAC'] = merged_df['AC'] / merged_df['Bayers'].replace(0, np.nan)
merged_df['AOV'] = merged_df['Gross Revenue'] / merged_df['Bayers'].replace(0, np.nan)
merged_df['APC'] = merged_df['Total Deals'] / merged_df['Bayers'].replace(0, np.nan)
merged_df['CLTV'] = merged_df['AOV'] * merged_df['APC']
merged_df['LTV'] = merged_df['Gross Revenue'] / merged_df['UA'].replace(0, np.nan)
merged_df['CM'] = (merged_df['CLTV'] - merged_df['CAC']) * merged_df['Bayers']

# Исключение строк с "Unknown" и "Web Developer Evening"
merged_df = merged_df[
    (merged_df['Product'] != 'Unknown') &
    (merged_df['Education Type'] != 'Unknown') &
    ~((merged_df['Product'] == 'Web Developer') & (merged_df['Education Type'] == 'Evening'))
]

# Сортировка данных
merged_df_sorted = merged_df.sort_values(by='Bayers', ascending=False)

# Преобразование категориальных столбцов в строки
merged_df_sorted['Product'] = merged_df_sorted['Product'].astype(str)
merged_df_sorted['Education Type'] = merged_df_sorted['Education Type'].astype(str)

# Функция для форматирования чисел
def format_number(value, col):
    if pd.isna(value):
        return ''
    if isinstance(value, (int, float)):
        if col == 'C1':
            return f'{value:.2f}%'  # Форматируем C1 как процент с двумя знаками после запятой
        elif col == 'APC':
            return f'{value:.2f}'
        elif 'Revenue' in col or col in ['CAC', 'AOV', 'CLTV', 'LTV', 'CM', 'AC', 'CPA']:
            if value >= 1e6:
                return f'€{value/1e6:.1f}M'
            elif value >= 1e3:
                return f'€{value/1e3:.1f}K'
            else:
                return f'€{value:.0f}'
        else:
            if value >= 1e6:
                return f'{value/1e6:.1f}M'
            elif value >= 1e3:
                return f'{value/1e3:.1f}K'
            else:
                return f'{value:.0f}'
    return str(value)

# Создание таблицы с помощью plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=list(merged_df_sorted.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[merged_df_sorted[col].apply(lambda x: format_number(x, col)) for col in merged_df_sorted.columns],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(title='Product and Education Type Performance Metrics')
fig.show()

# Создание визуализаций для ключевых метрик
metrics_to_plot = ['C1', 'CAC', 'AOV', 'CLTV', 'LTV']
fig = make_subplots(rows=len(metrics_to_plot), cols=1,
                    subplot_titles=metrics_to_plot,
                    vertical_spacing=0.1)

# Создаем новые метки с переносом строки
x_labels = [f"{product}<br>{education}" for product, education in zip(merged_df_sorted['Product'], merged_df_sorted['Education Type'])]

for i, metric in enumerate(metrics_to_plot, 1):
    fig.add_trace(
        go.Bar(x=x_labels,
               y=merged_df_sorted[metric],
               name=metric,
               text=[format_number(val, metric) for val in merged_df_sorted[metric]],
               textposition='auto'),
        row=i, col=1
    )
    fig.update_yaxes(title_text=metric, row=i, col=1)
    if metric == 'C1':
        fig.update_yaxes(ticksuffix='%', row=i, col=1)  # Добавляем % к меткам оси Y для C1

fig.update_layout(height=300*len(metrics_to_plot), width=1000,
                  title_text="Key Metrics by Product and Education Type")
fig.show()

# Создание тепловой карты для корреляции между метриками
correlation_matrix = merged_df_sorted[metrics_to_plot].corr()

fig = go.Figure(data=go.Heatmap(
                z=correlation_matrix.values,
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                colorscale='Viridis',
                zmin=-1, zmax=1))

fig.update_layout(title='Correlation Matrix of Key Metrics',
                  width=600, height=600)
fig.show()

# Вывод дополнительной информации
print("\nAdditional Insights:")
for col in metrics_to_plot:
    print(f"\n{col}:")
    print(f"  Mean: {format_number(merged_df_sorted[col].mean(), col)}")
    print(f"  Median: {format_number(merged_df_sorted[col].median(), col)}")
    print(f"  Standard Deviation: {format_number(merged_df_sorted[col].std(), col)}")
    print(f"  Coefficient of Variation: {merged_df_sorted[col].std() / merged_df_sorted[col].mean() * 100:.2f}%")


Additional Insights:

C1:
  Mean: 30.50%
  Median: 25.14%
  Standard Deviation: 10.58%
  Coefficient of Variation: 34.67%

CAC:
  Mean: €1.3K
  Median: €1.1K
  Standard Deviation: €805
  Coefficient of Variation: 63.77%

AOV:
  Mean: €6.1K
  Median: €4.3K
  Standard Deviation: €3.1K
  Coefficient of Variation: 50.89%

CLTV:
  Mean: €6.1K
  Median: €4.3K
  Standard Deviation: €3.1K
  Coefficient of Variation: 50.89%

LTV:
  Mean: €1.7K
  Median: €1.6K
  Standard Deviation: €456
  Coefficient of Variation: 27.33%


** Выводы:**

**Digital Marketing (утренние курсы):**
Высокий LTV (€2.2K) и CLTV (€9.7K) указывают на высокую доходность клиентов.
Конверсия составляет 22.97%, что является неплохим показателем.
CAC (€426) и CPA (€98) — относительно низкие затраты на привлечение клиента.

**UX/UI Design (утренние курсы):**
LTV (€2.0K) и CLTV (€9.4K) тоже высокие, что делает продукт привлекательным.
Конверсия (21.10%) чуть ниже, чем у Digital Marketing, но по-прежнему хорошая.
CAC (€906) выше, что указывает на возможные резервы для снижения стоимости привлечения.

**Web Developer (утренние курсы):**
Самый низкий LTV (€1.1K) и CLTV (€4.2K) среди всех продуктов.
CAC (€1.1K) выше, что говорит о высокой стоимости привлечения клиентов, что может снижать общую рентабельность.

**Digital Marketing (вечерние курсы):**
Очень высокая конверсия (45.16%), но при этом LTV ниже, чем у утренних курсов.
CAC (€1.3K) значительно выше, что может говорить о неэффективности каналов привлечения для вечерних курсов.

**UX/UI Design (вечерние курсы):**
Самый высокий CLTV (€3.8K) среди вечерних курсов, что говорит о потенциале этого продукта.
CAC (€2.6K) — самый высокий показатель, что является проблемой и точкой для оптимизации.

**Web Developer (вечерние курсы):**
Были исключены по причине недостатка данных для анализа, так как количество пользователей (UA) и клиентов (Bayers) крайне низкое.

##2.Из юнит-экономики определить точки роста бизнеса.


 **1.** Фокус на утренние курсы: Утренние курсы демонстрируют более высокие показатели LTV и CLTV при меньших затратах на привлечение клиентов (CAC). Рекомендуется сосредоточить маркетинговые усилия на продвижении утренних курсов.

**2.** Развитие Digital Marketing: Этот курс показывает наилучшие результаты по соотношению доходности и затрат. Следует рассмотреть возможность масштабирования этого направления и применения успешных стратегий к другим курсам.

**3.** Улучшение UX/UI Design: Высокие показатели LTV и CLTV делают этот курс перспективным. Необходимо работать над снижением CAC, особенно для вечерних курсов, чтобы повысить общую рентабельность.

**4.** Пересмотр стратегии Web Developer: Низкие показатели LTV и высокий CAC указывают на необходимость оптимизации этого курса. Следует либо пересмотреть ценообразование, либо улучшить содержание курса для повышения его ценности.

**5.** Анализ вечерних курсов: Несмотря на высокую конверсию, вечерние курсы имеют более высокий CAC. Необходимо провести аудит маркетинговых каналов для снижения затрат на привлечение.

**6.** Исследование причин низкого спроса на вечерние курсы Web Developer: Требуется дополнительный анализ для понимания причин низкого интереса и принятия решения о целесообразности продолжения этого курса.

**7.** Кросс-продажи: Учитывая высокие показатели CLTV, стоит разработать стратегии для увеличения среднего чека и стимулирования покупки дополнительных курсов существующими клиентами.Например предоставлять 50% скудку на покупку второго курса.

**8.** Оптимизация маркетинговых расходов: Необходимо провести анализ эффективности различных маркетинговых каналов для снижения общего CAC, особенно для вечерних курсов.



##3.Понять дерево метрик для бизнеса.

Было создано 3D Дерево метрик, визуализирующие взаимосвязи между ключевыми показателями, такими как:

**1. Общая прибыль**
- Выручка от всех сделок(Total Sales)
- Общее количество клиентов(Bayers)
- Средний чек (AOV)
- Суммарная ценность всех клиентов(CLTV)

**1.2. Затраты**
- Маркетинговые расходы(AC)
- Операционные расходы
- стоимость привлечения лида(CPA)
- Количество продаж на клиента(APC)

**2. Эффективность привлечения клиентов**
- Конверсия (C1)
- Стоимость за лид (CPA)
- Количество лидов (UA)
- Стоимость привлечения клиента (CAC)

**3. Удержание клиентов**
- Коэффициент удержания
- Коэффициент оттока

**4. Эффективность продуктов**
- CLTV по продуктам
- Маржинальность по продуктам (CM)

**5. Эффективность каналов продаж**
- CAC по каналам
- Конверсия по каналам

**6. Качество обучения**
- Удовлетворенность студентов
- Процент трудоустройства выпускников
- Качественное содержание курса
- Профессиональные преподаватели

**7. Ключевые точки роста:**
- Оптимизация CAC для повышения рентабельности.
- Увеличение конверсии (C1) для роста объема продаж и выручки.
- Увеличение LTV для повышения долгосрочной доходности клиентской базы.


 🔆 Необходимо адаптировать дерево метрик по мере развития бизнеса и изменения рыночных условий.

In [ ]:
import networkx as nx
import plotly.graph_objects as go
import colorsys

def create_metric_tree_3d():
    # Создаем граф
    G = nx.Graph()

    # Добавляем узлы и связи
    edges = [
        ("Дерево метрик", "1. Общая прибыль"),
        ("Дерево метрик", "2. Эффективность привлечения"),
        ("Дерево метрик", "3. Удержание клиентов"),
        ("Дерево метрик", "4. Эффективность продуктов"),
        ("Дерево метрик", "5. Эффективность каналов"),
        ("Дерево метрик", "6. Качество обучения"),
        ("Дерево метрик", "7. Ключевые точки роста"),
        ("1. Общая прибыль", "Выручка"),
        ("1. Общая прибыль", "Затраты"),
        ("Выручка", "Total Sales"),
        ("Выручка", "Bayers"),
        ("Выручка", "AOV"),
        ("Выручка", "CLTV"),
        ("Затраты", "AC"),
        ("Затраты", "Операционные расходы"),
        ("Затраты", "CPA"),
        ("Затраты", "APC"),
        ("2. Эффективность привлечения", "C1"),
        ("2. Эффективность привлечения", "UA"),
        ("2. Эффективность привлечения", "CAC"),
        ("3. Удержание клиентов", "Коэффициент удержания"),
        ("3. Удержание клиентов", "Коэффициент оттока"),
        ("4. Эффективность продуктов", "CLTV по продуктам"),
        ("4. Эффективность продуктов", "CM"),
        ("5. Эффективность каналов", "CAC по каналам"),
        ("5. Эффективность каналов", "Конверсия по каналам"),
        ("6. Качество обучения", "Удовлетворенность студентов"),
        ("6. Качество обучения", "Процент трудоустройства"),
        ("6. Качество обучения", "Качество курса"),
        ("6. Качество обучения", "Профессиональные преподаватели"),
        ("7. Ключевые точки роста", "Оптимизация CAC"),
        ("7. Ключевые точки роста", "Увеличение конверсии"),
        ("7. Ключевые точки роста", "Увеличение LTV")
    ]

    G.add_edges_from(edges)

    # Задаем позиции узлов в 3D
    pos = nx.spring_layout(G, dim=3, k=0.5, iterations=50)

    # Создаем градиентную цветовую схему
    def get_color(distance):
        # Используем HSV цветовую модель для создания градиента
        hue = 0.7 - (distance * 0.7)  # от фиолетового к голубому
        saturation = 0.8
        value = 0.9
        rgb = colorsys.hsv_to_rgb(hue, saturation, value)
        return f'rgb({int(rgb[0]*255)},{int(rgb[1]*255)},{int(rgb[2]*255)})'

    # Вычисляем расстояния от корневого узла
    distances = nx.single_source_shortest_path_length(G, "Дерево метрик")
    max_distance = max(distances.values())

    # Создаем узлы
    node_trace = go.Scatter3d(
        x=[],
        y=[],
        z=[],
        text=[],
        mode='markers+text',
        hoverinfo='text',
        textposition='bottom center',
        marker=dict(
            showscale=False,
            size=[40 if node == "Дерево метрик" else 30 if node.startswith(tuple('1234567')) else 10 for node in G.nodes()],
            color=[get_color(distances[node]/max_distance) for node in G.nodes()],
            line_width=2
        )
    )

    # Создаем ребра
    edge_trace = go.Scatter3d(
        x=[],
        y=[],
        z=[],
        line=dict(width=2, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    for edge in G.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])
        edge_trace['z'] += tuple([z0, z1, None])

    for node in G.nodes():
        x, y, z = pos[node]
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace['z'] += tuple([z])
        node_trace['text'] += tuple([node])

    # Создаем фигуру
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D Business metrics tree',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        scene=dict(
                            xaxis=dict(showbackground=False, showticklabels=False, showspikes=False),
                            yaxis=dict(showbackground=False, showticklabels=False, showspikes=False),
                            zaxis=dict(showbackground=False, showticklabels=False, showspikes=False)
                        )
                    )
                )

    return fig

# Создаем и отображаем визуализацию
fig = create_metric_tree_3d()
fig.show()

## 4.Понять на какую метрику продукта они будут воздействовать и сформировать гипотезы.

⚡
**Метрика:** C1(Conversion Rate) для вечерних курсов

**Гипотеза:** Применение успешных маркетинговых стратегий утренних курсов Digital Marketing к вечерним курсам  позволит увеличит конверсию на 10%

⭕
**Нулевая гипотеза:** Внедрение успешных маркетинговых стратегий утренних курсов не приведет к увеличению конверси.

**Обоснование:** Большая часть дохода (54.9%) приносит Digital Marketing(Morning). Курсы по UX/UI Design (Morning) составляют 25.7% дохода, вечерние курсы значительно уступают утренним по всем направлениям.Учитывая высокий CLTV вечерних курсов UX/UI Design и успешный опыт привлечения клиентов на утренние курсы, перенос этих практик может значительно повысить эффективность привлечения клиентов на вечерние курсы.

**Меры**
Для разработки целевых стратегий, рекомендуется провести детальный анализ портфеля курсов. Этот анализ предоставит основу для стратегических решений, направленных на оптимизацию образовательных продуктов.
Учитывая что наиболее эффективным типом оплаты являются "Recurring Payments", что составляет коэффициент конверсии 71.2% анализировать процесс оплаты и нюансы связоный с местной бюракратией, что позволит ускорить сложный и длительный процесс оплаты.

---

⚡
**Метрика:** CAC (Customer Acquisition Cost) аудит маркетинговых каналов

**Гипотеза:** Оптимизация рекламных кампаниий, нацеленых на более релевантную аудиторию,снизит стоимость привлечения одного клиента (CAC) на 15%.

⭕
**Нулевая гипотеза:** Оптимизация рекламных кампаний не приведет к изменению стоимости привлечения одного клиента(CAC).

**Обоснование:** Высокий процент потерянных сделок 73%(стадия "Lost").В основной базе клиентов преобладаниют категории ("Non Qualified") 33% и ("Non Target")40% что указывает на неэффективность текущих методов в привлечения качественных лидов.

**Меры**
Для оптимизации и повышения рентабельности продаж, рекомендуется провести глубокую сегментацию клиентской базы, выявив характеристики наиболее перспективных клиентов (категории A и B). Это позволит сфокусировать маркетинговые усилия. Также прозрачность компании и эфективность контента транслироемое отделом продаж, может улудшить привлечения качественных лидов-например коифицент трудоустроиных выпускников основываясь на реальные результаты.




##5.Описать метод проверки гипотез с формулированием условия проведения гипотезы.



⚡
**Увеличение конверсии (C1) для вечерних курсов**
- Контрольная группа: Применение успешных маркетинговых стратегий утренних курсов к вечерним курсам.
- Тестовая группа: Придерживаться первоначальной стратегии и оставить все без изменений.

**KPI и метрики:**
- Внедрить систему ключевых показателей эффективности (KPI) для каждого этапа - воронки продаж, что позволит оперативно отслеживать улучшения и выявлять проблемные области.
- Сбор данных: Получение данных о конверсии вечерних курсов и количестве привлеченых лидов (UA), для обеих групп на протяжении тестового периода.


⏳
**Условия проведения теста**
- Длительность теста: 2 недели.(желательно-до 4-6 недель)
- Действие:Адаптация успешных маркетинговых стратегий утренних курсов для вечерних и настройка таргетинга рекламы на аудиторию, схожую со студентами утренних курсов
- Размер выборки:Минимальный размер выборки: не менее 100 уникальных посетителей во всех вечерних группах.
Оптимальный размер выборки: 300-500 уникальных посетителей во всех вечерних группах.
- Ожидаемый прирост:Увеличение конверси на 10% и количества лидов (UA) по сравнению с утреними курсами.
- Проведение t-теста:Сравнение средних значений C1 между двумя группами для определения статистической значимости различий.
Желаемый уровнь статистической значимости (α = 0.05)
Желаемый статистической мощности теста (1-β = 0.8)


---

⚡
**Снижение стоимости привлечения одного клиента (CAC)**
- Контрольная группа: Рекламные кампании, не использующие сегментацию и персонализацию.
- Тестовая группа: Рекламные кампании, использующие сегментацию и персонализацию.

**A/B тест**
- Определение целевой метрики: Стоимость привлечения одного клиента (CAC).
- Разделение аудиторий: Случайное распределение рекламных кампаний в контрольную и тестовую группы.
- Сбор данных: Получение данных о затратах на привлеченых лидов (UA) и количестве привлеченных клиентов для обеих групп на протяжении тестового периода.


⏳
**Условия проведения теста**
- Длительность теста: 2 недели.(желательно-до 4-6 недель)
- Действие: Фокусироваться на наиболее эффективных каналах и кампаниях c учетом сегментации и персонализации.
- Размер выборки: 1000 клиентов в каждой группе (контрольная и тестовая).
- Ожидаемое снижение 15% в CAC тестовой группы по сравнению с контрольной группой.






In [ ]:
# Установка начального значения для генератора случайных чисел
np.random.seed(15)

# Пример данных
control_group = np.random.normal(loc=50, scale=5, size=1000)
test_group = np.random.normal(loc=55, scale=5, size=1000)

# Создание графика
fig = go.Figure()

# Добавление гистограмм
fig.add_trace(
    go.Histogram(x=control_group, name="Контрольная группа", opacity=0.75)
)
fig.add_trace(
    go.Histogram(x=test_group, name="Тестовая группа", opacity=0.75)
)

# Обновление макета
fig.update_layout(
    title="Анализ результатов A/B теста",
    xaxis_title="Значение",
    yaxis_title="Частота",
    barmode='overlay',
    height=500,  # Уменьшена высота
    width=1000,  # Увеличена ширина
    legend=dict(
        title="Группы",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Отображение графика
fig.show()

##Интерпретация A/B t-теста:

**Статистическая значимость:**
Визуальный анализ гистограммы показывает, что распределения значений для контрольной и тестовой групп значительно отличаются.
Пик частоты в контрольной группе наблюдается около значения 50, тогда как в тестовой группе — около значения 55.
Тестовая группа имеет большее количество участников с высокими значениями (от 55 до 65), что указывает на более высокие средние значения по сравнению с контрольной группой.

**Вывод:**
На основании визуального анализа гистограммы мы можем сделать вывод, что тестовая группа показала более высокие значения  по сравнению с контрольной группой.
Это указывает на то, что изменения  рекламных кампаниий, нацеленых на более релевантную аудиторию,снизит стоимость привлечения одного клиента .


---


##Анализ результатов T теста

In [ ]:
# Установка начального значения для генератора случайных чисел
np.random.seed(15)

# Пример данных
control_group = np.random.normal(loc=50, scale=5, size=1000)   # Среднее значение = 50, стандартное отклонение = 5
test_group = np.random.normal(loc=55, scale=5, size=1000)      # Среднее значение = 55, стандартное отклонение = 5

# Проведение t-теста для независимых выборок
t_stat, p_value = stats.ttest_ind(control_group, test_group)

print(f"t-статистика: {t_stat}")
print(f"p-значение: {p_value}")

# Интерпретация результатов
if p_value < 0.05:
    print("Отклоняем H0, есть статистически значимая разница между группами.")
else:
    print("Не можем отклонить H0, статистически значимой разницы между группами нет.")

t-статистика: -22.382436864898303
p-значение: 3.3939267040934477e-99
Отклоняем H0, есть статистически значимая разница между группами.


##Интерпретация t-теста:##
**Статистическая значимость**:
- Значение p-значения (3.3939267040934477e-99) значительно меньше порогового уровня 0.05.
- Это означает, что мы можем отклонить нулевую гипотезу.

**Размер эффекта:**
t-статистика (-22.382436864898303) указывает на большую разницу между группами. Отрицательное значение говорит о том, что среднее значение тестовой группы выше, чем контрольной.

**Вывод:**
На основании результатов t-теста мы отклоняем нулевую гипотезу, что указывает на наличие статистически значимой разницы между контрольной  и тестовой  группами. Это подтверждает, что применение маркетинговых стратегий утренних курсов для вечерних и настройка таргетинга рекламы на аудиторию, схожую со студентами утренних курсов, действительно приводит к увеличению конверсии..

---
❗*Важно отметить, что эти результаты основаны на симулированных данных. В реальном эксперименте необходимо учитывать возможные внешние факторы, которые могут повлиять на результаты.*



